In [1]:
import cplex
import networkx as nx

In [2]:
graph_names = ["brock200_1.txt", "brock800_4.txt", 
               "c-fat200-5.txt", "c-fat500-10.txt", "hamming10-2.txt", "hamming10-4.txt",
               "keller4.txt", "keller5.txt", "MANN_a27.txt",
               "san200_0.7_1.txt", "sanr400_0.7.txt"]

In [3]:
graphs_for_testing = []

for graph in graph_names:

    with open("./graphs/"+graph) as file:
        data = file.readlines()
    data = data[1:]
    
    G = nx.Graph()

    for line in data:
        line = line.strip().split()
        u, v = int(line[1])-1, int(line[2])-1
        G.add_edge(u, v)
        
    graphs_for_testing.append((graph, G))

In [6]:
results = []

In [8]:
for g in graphs_for_testing[:-1]:
    
    prob = cplex.Cplex()
    prob.set_problem_name("Minimum Vertex Cover")
    
    prob.set_problem_type(cplex.Cplex.problem_type.LP)

    prob.objective.set_sense(prob.objective.sense.minimize)

    n = len(g[1].nodes)

    names = [str(i) for i in range(n)]

    # Objective (linear) weights
    w_obj = [1 for i in range(n)]
    # Lower bounds
    low_bnd = [0 for i in range(n)]
    # Upper bounds
    upr_bnd = [1 for i in range(n)]
    prob.variables.add(names=names, obj=w_obj, lb=low_bnd, ub=upr_bnd)

    # Set variable types
    all_int = [(var, prob.variables.type.integer) for var in names]
    prob.variables.set_types(all_int)

    constraints = []
    for e in g[1].edges:
        constraints.append([[str(e[0]), str(e[1])], [1, 1]])

    # Constraint names
    constraint_names = ["".join(x[0]) for x in constraints]

    # Each edge must have at least one vertex
    rhs = [1] * len(constraints)

    # G -> >=
    constraint_senses = ["G"] * len(constraints)

    # And add the constraints
    prob.linear_constraints.add(names=constraint_names,
                                lin_expr=constraints,
                                senses=constraint_senses,
                                rhs=rhs)

    #Time limit
    prob.parameters.timelimit.set(180*60)

    # Solve the problem
    prob.solve()
    
    print("Solution result is: %s" % prob.solution.get_status_string())
    print(prob.solution.get_values())
    
    results.append((g[0], sum(prob.solution.get_values())))

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               10800
Found incumbent of value 200.000000 after 0.00 sec. (0.09 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 2938 rows and 0 columns.
MIP Presolve modified 2128 coefficients.
Reduced MIP has 2128 rows, 200 columns, and 9002 nonzeros.
Reduced MIP has 200 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.06 sec. (17.49 ticks)
Probing time = 0.00 sec. (0.69 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 2128 rows, 200 columns, and 9002 nonzeros.
Reduced MIP has 200 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (9.43 ticks)
Probing time = 0.01 sec. (0.69 ticks)
Clique table members: 2128.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.20

  38515 23474      173.8552    95      179.0000      172.5263  2431106    3.62%
  39504 24029      177.1729    69      179.0000      172.5842  2482923    3.58%
  40556 24756      175.3547    84      179.0000      172.6523  2557008    3.55%
  41569 25224      175.4218    86      179.0000      172.7193  2601007    3.51%
Elapsed time = 111.42 sec. (56053.26 ticks, tree = 13.63 MB, solutions = 8)
  42645 25915      175.8253    79      179.0000      172.7905  2666574    3.47%
  43647 26545      175.5991    74      179.0000      172.8615  2726720    3.43%
  44681 27019      177.2308    72      179.0000      172.9262  2777282    3.39%
  45724 27661      175.6469    73      179.0000      173.0038  2844961    3.35%
  46767 27938        cutoff            179.0000      173.0721  2871747    3.31%
  47779 28667      177.4203    69      179.0000      173.1282  2945185    3.28%
  48787 29159      177.0404    69      179.0000      173.1919  2997742    3.24%
  49827 29552      177.9062    65      179.0

      0     0      700.4757   800      783.0000    Cliques: 5     6810   10.54%
      0     0      700.6069   800      783.0000    Cliques: 8     7422   10.52%
      0     0      700.7219   800      783.0000   Cliques: 13     8103   10.51%
      0     0      700.8425   800      783.0000    Cliques: 6     8697   10.49%
      0     0      700.9346   800      783.0000    Cliques: 8     9158   10.48%
      0     0      701.0168   800      783.0000    Cliques: 3     9624   10.47%
      0     0      701.0560   800      783.0000    Cliques: 4    10002   10.47%
Heuristic still looking.
Detecting symmetries...
      0     2      701.0560   800      783.0000      701.0560    10002   10.47%
Elapsed time = 726.37 sec. (333386.91 ticks, tree = 0.02 MB, solutions = 3)
      1     3      701.2043   799      783.0000      701.0572    11819   10.47%
      2     3      721.7541   525      783.0000      701.0572    22777   10.47%
      3     5      722.0777   521      783.0000      701.2044    25317   10

   1233     7      736.3864   355      782.0000      703.4082   397226   10.05%
   1234     8      736.5274   355      782.0000      703.4082   397492   10.05%
   1235     9      736.6341   354      782.0000      703.4082   398079   10.05%
   1239    12      761.8237   142      782.0000      703.4082   400850   10.05%
   1257    30      776.2653    51      782.0000      703.4082   403264   10.05%
   1291    47      751.0919   220      782.0000      703.4082   405716   10.05%
   1297    52      751.4474   215      782.0000      703.4082   408124   10.05%
   1304    59      762.7085   136      782.0000      703.4082   410287   10.05%
   1315    72      762.5943   130      782.0000      703.4082   424495   10.05%
Elapsed time = 2736.08 sec. (1077572.45 ticks, tree = 5.52 MB, solutions = 4)
   1332    89      763.3906   125      782.0000      703.4082   427170   10.05%
   1354   111      774.3925    66      782.0000      703.4082   430157   10.05%
   1380     4      703.5603   798      782

   6319  4860      726.5637   480      782.0000      705.2381  1637890    9.82%
   6404  4913      739.1113   355      782.0000      705.2381  1645524    9.82%
   6529  4965      745.4346   300      782.0000      705.2381  1653081    9.82%
   6683  5080      737.7659   364      782.0000      705.2381  1680437    9.82%
   6864  5216      736.8907   378      782.0000      705.2381  1701972    9.82%
Elapsed time = 3708.00 sec. (1351735.21 ticks, tree = 385.83 MB, solutions = 4)
   6950  5403      742.1173   336      782.0000      705.2381  1735307    9.82%
   7054  5531      725.0729   495      782.0000      705.3178  1753213    9.81%
   7096  5555      727.3718   470      782.0000      705.4262  1758521    9.79%
   7176  5682      728.9615   454      782.0000      705.5389  1787143    9.78%
   7244  5750      752.5559   212      782.0000      705.5389  1815286    9.78%
   7396  5898      734.3255   396      782.0000      705.5389  1838728    9.78%
   7506  5936      739.2658   357      7

...

 219577 190675      746.5000   172      749.0000      745.0000 48354804    0.53%
 219851 190752      746.9464   159      749.0000      745.0000 48371885    0.53%
 220210 191159      747.1378   148      749.0000      745.0000 48484047    0.53%
 220559 191132        cutoff            749.0000      745.0000 48452504    0.53%
 220955 191771      747.3048   140      749.0000      745.0000 48627282    0.53%
 221450 192036      746.5366   133      749.0000      745.0000 48697075    0.53%
 221873 191984      747.3324   129      749.0000      745.0000 48666078    0.53%
Elapsed time = 6672.29 sec. (3038184.30 ticks, tree = 5243.93 MB, solutions = 7)
Nodefile size = 3181.22 MB (3170.93 MB after compression)
 222246 192986      746.9834   156      749.0000      745.0000 48904702    0.53%
 222587 193330      746.6833   156      749.0000      745.0000 48966528    0.53%
 222948 193364      748.0000   129      749.0000      745.0000 48984854    0.53%
 223268 193563      746.0345   144      749.0000   

Nodefile size = 3839.06 MB (3824.08 MB after compression)
 254017 219018      745.8058   173      749.0000      745.0460 54680790    0.53%
 254425 218829      747.0363   164      749.0000      745.0460 54642483    0.53%
 254897 219545      746.9602   144      749.0000      745.0468 54803160    0.53%
 255360 219654      746.2256   155      749.0000      745.0485 54822865    0.53%
 255506 219852      747.7037   132      749.0000      745.0485 54857180    0.53%
 255664 220602      746.5780   143      749.0000      745.0485 55012738    0.53%
 255877 220653      746.3547   163      749.0000      745.0485 55015272    0.53%
 256334 220834      747.3518   140      749.0000      745.0503 55051850    0.53%
 256735 221456      745.9102   156      749.0000      745.0512 55179687    0.53%
 257258 221251      746.4386   119      749.0000      745.0512 55150063    0.53%
Elapsed time = 7426.79 sec. (3400522.22 ticks, tree = 5972.60 MB, solutions = 7)
Nodefile size = 3912.88 MB (3897.23 MB after compre

 296398 253375        cutoff            749.0000      745.1050 62472982    0.52%
 297940 254443      747.2641   137      749.0000      745.1060 62692525    0.52%
 299500 255940        cutoff            749.0000      745.1088 63059117    0.52%
Elapsed time = 8338.08 sec. (3836419.11 ticks, tree = 6825.37 MB, solutions = 7)
Nodefile size = 4767.02 MB (4745.12 MB after compression)
 301348 257352      746.4847   131      749.0000      745.1117 63363279    0.52%
 303131 258632      745.5732   183      749.0000      745.1139 63629008    0.52%
 305120 260422      747.8014   131      749.0000      745.1165 63976377    0.52%
 306903 261878      747.3927   150      749.0000      745.1192 64242571    0.52%
 308385 263563      748.0000   139      749.0000      745.1214 64614067    0.52%
 309744 263852      747.1467   141      749.0000      745.1231 64725856    0.52%
 311257 265055      747.5299   148      749.0000      745.1250 65038882    0.52%
 311506 265471      748.0000   127      749.0000   

 391439 17239      745.6667   175      749.0000      745.1857 81068528    0.51%

Clique cuts applied:  47

Root node processing (before b&c):
  Real time             =   38.40 sec. (17908.24 ticks)
Parallel b&c, 4 threads:
  Real time             = 10761.62 sec. (4995672.14 ticks)
  Sync time (average)   =  301.26 sec.
  Wait time (average)   =    2.83 sec.
                          ------------
Total (root+branch&cut) = 10800.03 sec. (5013580.37 ticks)
Solution result is: time limit exceeded
[1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               10800
Found incumbent of value 200.000000 after 0.00 sec. (0.10 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 4310 rows and 0 columns.
MIP Presolve modified 1659 coefficients.
Reduced MIP has 1660 rows, 200 columns, and 8951 nonzeros.
Reduced MIP has 200 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.05 sec. (18.53 ticks)
Probing time = 0.00 sec. (0.77 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 1660 rows, 200 columns, and 8951 nonzeros.
Reduced MIP has 200 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (8.63 ticks)
Probing time = 0.00 sec. (0.77 ticks)
Clique table members: 1660.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.13

In [9]:
results

[('brock200_1.txt', 179.0),
 ('brock800_4.txt', 782.0),
 ('c-fat200-5.txt', 142.0),
 ('c-fat500-10.txt', 374.0),
 ('hamming10-2.txt', 512.0),
 ('hamming10-4.txt', 984.0),
 ('keller4.txt', 160.0),
 ('keller5.txt', 749.0),
 ('MANN_a27.txt', 252.0),
 ('san200_0.7_1.txt', 170.0)]